# HPC intro

## Using R

R is of course a very useful programming language for data processing, analysis and visualization.

There are many tutorials and courses that will teach you R, so that is not the scope of this tutorial.  Here you will learn how to run R scripts on our HPC infrastructure, assuming you are already familiar with the language and packages.

### R scripts

You may be used to executing R commands line by line in RStudio, if so, forget about that approach.  It is (perhaps, probably not) fine while you are developing a new R script, but that approach is inherently interactive, and hence doesn't lend itself well to HPC systems, where most scripts are executed in batch mode, i.e., without any user interaction.  You will need to think in terms of R scripts that are completely executed from top to bottom.

Given that R scripts are simple text files, you can create or modify them using your favorite editor.  You can do this for instance on the infrastructure using `nano`, or on your own system and transfer the finished script or library to the HPC system.

To build up gradually, you can start with a very simple script that takes a string as a command line argument and prints a greeting to standard output.  Your script is stored in a file `hello.R` which could look like this.

```R
#!/usr/bin/env Rscript

args = commandArgs(trailingOnly=TRUE)

if (length(args) != 1) {
        stop('a name is required')
}
cat('Hello ', args[1], '!', sep='')
```

This is of course a very simple R script, but it shows you how to handle command line arguments.  The vector `args` contains string values, so if you want to pass numerical arguments via the command line, you would have to convert these strings to numerical values, e.g.,
```R
x <- as.double(args[2])
n <- as.integer(args[3])
```

**Important:** when you use scripts that you created on a Windows computer, remember to check the file names.  If they contain `C:\` or similar, make sure to modify them to relative directories!

### Running simple scripts

An HPC system is almost by definition a multi-tenant system.  The users on such a system have specific requirements with respect to the software they want to use.  For instance, some may want to work with a certain version of Python, while others prefer a newer one.

To deal with this, most HPC system use a module system that allows to easily pick the software and its specific version you want to use.  There is just a single command to interact with the software stack: `module`.  It has several subcommand that you will learn about belo.

#### Available software

In order to get a list of the software that is available through the module system, you can use the `module available` command.  That will list all the software packages that you can use on the system.

Since this list is huge, you can be a bit more specific by providing (part of) the name of the software package you are looking for.  Note that this is case-sensitive.  Since you only want to see the available  versions of R, you can use the `-r` option to specify a regular expression as a search pattern.  The regular expression is `'^R/'`, which means that you select software packages that start (`^`) with capital R, and that are followed by a `'/`'` which separates the package name from its version.

In [ ]:
module -r available '^R/'

To run R, the `R/4.1.0-foss-2021a` module sounds promising.  The name may seem a bit cryptic, but once you understand the pattern, it is easy to interpret.

The name of a module consists of several parts that provide useful information:
  * `R` is the name of the software package;
  * `4.1.0` is the version of that package, i.e., of the R distribution;
  * `foss-2021a` tells you that this R distribution has been compiled using the foss toolchain version 2021a,
    i.e., the GCC compiler suite, OpenMPI and OpenBLASS/Lapack/FFTW3.

#### Using a software package

To use a software package, you simply load the corresponding module.

In [ ]:
module load R/4.1.0-foss-2021a

You can verify that you now use the Python interpreter you expect by checking version and location of the `python` executable.

In [ ]:
R --version

In [ ]:
which R

Now you can run your script using this version of Python.

In [ ]:
Rscript hello.R "wonderful world of HPC"

Note that since you want to pass the string "wonderful world of HPC" as a single argument to your script, you have to quote it.

There is a lot more to learn about the module system, and you are likely to need that at some point.  Perhaps it would be a good idea to check out the [tutorial](003_software_modules.ipynb) now.

## R packages

Of course, R by itself is not that useful, you like want to use multiple R packages from CRAN.  Some may have been preinstalled for you, but likely not all.  If so, no worries, you can (sort of) easily install them yourself.

Click to watch the [**video**](https://youtu.be/jMZIeh3RA5s).

However, life is sometimes a bit more complicated.  Some R packages depend on software packages that should be installed on your system.  For instance, as you will see in the next video, the R package `gsl` relies on the GNE GSL (GNU Scientific Library) package, which itself has some other dependencies.

Click to watch the [**video**](https://youtu.be/NcPeRuqhUS4).

Remember (you did the [tutorial]() on the module system, right?) that you can search for GSL.

In [ ]:
module spider GSL

Once you have installed the gsl R package as described in the video, you can try it using the following R script.  The script will plot the Airy function (defined in `gsl`) and create an EPS file that you can include in a $\LaTeX$ document.

```R
#!/usr/bin/env Rscript

# script to plot the Airy function
# https://en.wikipedia.org/wiki/Airy_function
# the function values are computed using the GNU Scientific Library

# Load the ggplot2 and gsl libraries
library(ggplot2)
library(gsl)

# Create a data frame, the x-range runs from 0.0 to 2.0
df <- data.frame(x = seq(0.0, 2.0, length.out = 100))

# compute the Airy function values
df$y <- airy_Ai(df$x)

# Create a ggplot object
p <- ggplot(df, aes(x = x, y = y)) + geom_point()

# Save the plot to a file
ggsave("airy.eps", plot = p, width = 4, height = 4, dpi = 300)
```

Installing R packages in this way will work very often, but unfortunately not always.  Some have rather arcane depencies that make them very hard to install on an HPC system.

If you have trouble, please [contact support](https://docs.vscentrum.be/en/latest/user_support.html).

## Summary

In this tutorial you learned how to run R scripts using an R module.
  
For the module system, You learned how to
  * list available modules using `module available`
  * search for modules using `module spider`
  * use the software package using `module load`

You also learned how to install your own R packages.

## Where to go from here?

You can now run an R script on the login node, but that is only useful for very short computation, i.e., scripts that run in a minute or less.  You share the login node with many other users of the HPC system, so if you perform computationally intensive computations on this system, it will impact the performance for all other users.

Your real workloads will run on the compute nodes of the HPC system, and these computations are typically run via a job script.  You can learn more about that in
  * [job scripts and the scheduler](020_jobs.ipynb).